# Coursera_DataScientist_Course9_Week3_Assignment_HSD_20190612 (Part 2 of 3)

### COMMENTS: In this notebook I have documented the objective and the steps indicated in the assignment in order to follow step by step the instructions and keep track of the activities step by step

### IMPORTANT:  This is an incremental file. I took file "HSD_Week3_Part_1_of_3"  as a basis, and added below the code for Part 2 of 3


### =================================  PART 1 OF 3 =====================================

### OBJECTIVE OF THE ACTIVITY: Explore and cluster the neighborhoods in Toronto. 

### STEP 1:  Create a new Notebook for this assignment.   ---> STATUS: step completed. It is this notebook

### STEP 2: Build the code to scrape the  Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,   -> STATUS: completed. See the code below

In [2]:
#import the libraries
import numpy as np   # import library to managed daa in vectorized way
import pandas as pd  # library for data analysis
import requests      # library for requests

In [3]:
#load the html file from internet to a local file for later processing with Beautiful Soup library
!wget -q -O 'file_with_input_info' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [4]:
#import the library that will help to get the contents from html file
from bs4 import BeautifulSoup

In [5]:
#read the content of the html file using de BS library
with open("file_with_input_info") as html_file:
    soup = BeautifulSoup(html_file,'lxml')
#print(soup.prettify())     #this line is commented. I activated when testing the code in order to have the look and feel of the content

In [6]:
#store the table in a variable in order to scan contents at level table / body / line / field
my_table = soup.find('table')
#print(my_table) #this line is commented. I activated when testing the code in order to have the look and feel of the content
my_body = my_table.tbody
#print(my_body) #this line is commented. I activated when testing the code in order to have the look and feel of the content

In [7]:
# With this loop the table is recognized as a whole
#Create an empty dataframe in order to store the information taht will be read from input source
my_df = pd.DataFrame(columns=('PostalCode','Borough','Neighborhood'))
# Need special treatment (try & except) to isolate records with <th> labels from <td> labels that are the ones taht have the content
my_counter_of_records = 0
my_fields_content = [' ',' ',' ']
for my_table_record in my_table.tbody:
    my_list = ""
    my_ok_to_save = False
    try:
        my_field_index = 0
        for my_field in my_table_record.find_all('td'):
            my_fields_content[my_field_index] = my_field.text
            if my_field_index == 2:
                my_fields_content[my_field_index] = my_fields_content[my_field_index][:-1]    #take out "\n" char at the end of line (in 3rd field)
            my_list = my_list + "," + my_field.text
            my_field_index = my_field_index + 1 
            my_ok_to_save = True
    except:
        my_counter_of_records = my_counter_of_records + 0   # counter remains the same. this is for making it explicit
    if my_ok_to_save == True:
        # print(my_counter_of_records, my_list)             #line only used when testing the code
        my_df.loc[len(my_df)] = my_fields_content           #inserts fields in the record of the dataframe
        my_counter_of_records = my_counter_of_records + 1   #increments the record number for next iteration   
        

In [8]:
my_df.shape

(288, 3)

In [9]:
my_df.head(5)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### STEP 3: Create the dataframe

#### Step 3.1 The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood  --> STATUS DONE.  It was done at the ned of step 2

#### Step 3.2 Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.  --> STATUS DONE. See code below

In [10]:
print(my_df.shape)

(288, 3)


In [11]:
my_df2 = pd.DataFrame(columns=('PostalCode','Borough','Neighborhood'))
my_df2 = my_df[my_df['Borough']!='Not assigned']     #copy data in the new df only lines with Borough different from 'Not assigned' value
my_df2.reset_index(drop=True,inplace=True)

In [12]:
print(my_df2.shape)

(211, 3)


#### Step 3.3 More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma  --> STATUS DONE. See code below

In [13]:
#just checking the example mentioned in the exercise instructions..  It is true, there are more than 1 Neighborhood per PostaCode
print(my_df2[my_df2['PostalCode']=='M5A'])

  PostalCode           Borough  Neighborhood
2        M5A  Downtown Toronto  Harbourfront
3        M5A  Downtown Toronto   Regent Park


In [14]:
#Now we will set up a process to combine the Neighborhood names that are associated with the same PostCode
my_df3 = pd.DataFrame(columns=('PostalCode','Borough','Neighborhood'))
for my_i in range(0,len(my_df2)):
    my_df2_postal_code = my_df2.loc[my_i][0]          #capture the postalcode from the original dataframe 
    postalcode_is_new = True                          #assumes it will be a new code in the destination dataframe
    for my_j in range (0,len(my_df3)):                #loop to determine if PostalCode is new in destination dataframe
        my_df3_postal_code = my_df3.loc[my_j][0]
        my_df3_neighborhood = my_df3.loc[my_j][2]
        if my_df2_postal_code == my_df3_postal_code:
            postalcode_is_new = False
            my_df3_INDEX = my_j                       #saves the reference to the line in the destination dataframe, the one that matched
    if postalcode_is_new == True:                     #if it is a new item, it is added into the destination dataframe
        my_df3_postal_code =  my_df2.loc[my_i][0]
        my_df3_borough =  my_df2.loc[my_i][1]
        my_df3_neighborhood =  my_df2.loc[my_i][2]
        my_df3.loc[len(my_df3)] = [my_df3_postal_code, my_df3_borough, my_df3_neighborhood]
    else:                                             #it it is not a new item, checks it combines the content of neighborhood
        my_df3_neighborhood = my_df2.loc[my_i][2] + ", " +  my_df3_neighborhood
        my_df3.loc[my_df3_INDEX][2] = my_df3_neighborhood        

In [15]:
print(my_df3.shape)

(103, 3)


In [16]:
#just re-checking the example mentioned in the exercise instructions..  It is true, there are more than 1 Neighborhood per PostaCode
print(my_df3[my_df3['PostalCode']=='M5A'])

  PostalCode           Borough               Neighborhood
2        M5A  Downtown Toronto  Regent Park, Harbourfront


In [17]:
my_df3.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Not assigned


#### Step 3.4 If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.    --> STATUS DONE. See code below

In [18]:
#Check the cases indicated as cases to be fixed
for my_i in range(0,len(my_df3)):
    my_df3_neighborhood = my_df3.loc[my_i][2]         #capture the neighborhood
    if my_df3_neighborhood == 'Not assigned':
        my_df3_postal_code =  my_df3.loc[my_i][0]
        my_df3_borough =  my_df3.loc[my_i][1]
        my_df3_neighborhood =  my_df3.loc[my_i][2]
        print("record to be transformed: ", my_df3_postal_code," , ", my_df3_borough, " , ", my_df3_neighborhood)

record to be transformed:  M7A  ,  Queen's Park  ,  Not assigned


In [19]:
#Transform the record..
for my_i in range(0,len(my_df3)):
    my_df3_neighborhood = my_df3.loc[my_i][2]         #capture the neighborhood
    if my_df3_neighborhood == 'Not assigned':
        my_df3_postal_code =  my_df3.loc[my_i][0]
        my_df3_borough =  my_df3.loc[my_i][1]
        my_df3_neighborhood =  my_df3.loc[my_i][2]
        my_df3.loc[my_i][2] = my_df3.loc[my_i][1]     #transform the record

In [20]:
#re-Check the cases indicated as cases to be fixed
for my_i in range(0,len(my_df3)):
    my_df3_neighborhood = my_df3.loc[my_i][2]         #capture the neighborhood
    if my_df3_neighborhood == 'Not assigned':
        my_df3_postal_code =  my_df3.loc[my_i][0]
        my_df3_borough =  my_df3.loc[my_i][1]
        my_df3_neighborhood =  my_df3.loc[my_i][2]
        print("record to be transformed: ", my_df3_postal_code," , ", my_df3_borough, " , ", my_df3_neighborhood)

#### Step 3.5 Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.   --> STATUS: DONE. See code above. I have added comments

#### Ste 3.6 In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe. --> STATUS: DONE. See code below

In [21]:
my_df3.shape

(103, 3)

In [22]:
my_df3.head(103)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### STEP 4: Submit link to your Notebook on the Github repository  --> STATUS: Done


I added the link of this notebook in the coursera instruction section

### =================================  PART 2 OF 3 =====================================

#### STEP A   
... Geocoder Python package ... The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data   --> STATUS: DONE. See code below

In [45]:
# I use the cvs file to add latirud and longitud to the dataframe created in part 1 o 3 above.

In [46]:
my_dfLL = pd.read_csv(r'http://cocl.us/Geospatial_data') 

In [47]:
my_dfLL.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [48]:
#add 2 columns to destination dataframe
my_df3['Latitude'] = " "
my_df3['Longitude'] = " "

In [49]:
#Now we will set up a process to add on the orignal dataframe the content of Long and Lat based on PostalCode
for my_i in range(0,len(my_df3)):
    my_df3_postal_code = my_df3.loc[my_i][0]          #capture the postalcode from the original dataframe 
    LL_is_found = False                               #assumes there is not LL code in the LL dataframe
    for my_j in range (0,len(my_dfLL)):               #loop to determine if LL is found in LL dataframe
        my_dfLL_postal_code = my_dfLL.loc[my_j][0]
        if my_df3_postal_code == my_dfLL_postal_code:
            LL_is_found = True
            my_dfLL_INDEX = my_j                      #saves the reference to the line in the destination dataframe, the one that matched
            my_latitud = my_dfLL.loc[my_j][1]
            my_longitud = my_dfLL.loc[my_j][2]
    if LL_is_found == True:                           #if it was found the LL, it is added into the original dataframe
        my_df3.loc[my_i][3] = my_latitud
        my_df3.loc[my_i][4] = my_longitud

In [50]:
my_df3.head(103)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
4,M7A,Queen's Park,Queen's Park,43.6623,-79.3895
5,M9A,Etobicoke,Islington Avenue,43.6679,-79.5322
6,M1B,Scarborough,"Malvern, Rouge",43.8067,-79.1944
7,M3B,North York,Don Mills North,43.7459,-79.3522
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7064,-79.3099
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789


#### Step B: Once you are able to create the above dataframe, submit a link to the new Notebook on your Github repository. (2 marks) --> STATUS: DONE. 

I added the link of this notebook in the coursera instruction section